# Tutorial: Perturbation of the Neoclassical Growth Model

## Advanced Macro: numerical methods

__Warm-up: install and test the `ForwardDiff` library. Check the jacobian function.__

In [4]:
#using Pkg; Pkg.add("ForwardDiff")
using ForwardDiff: jacobian

    Updating registry at `~/.julia/registries/General`


    Updating 

git-repo `https://github.com/JuliaRegistries/General.git`


   Resolving package versions...


┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36


...

  ✓ PlotlyJS


  ✓ RecipesPipeline


  ✓ PlotThemes


  ✓ Plots


  15 dependencies successfully precompiled in 55 seconds (232 already precompiled)


__Warm-up(2): install and test the `NLSolve` library. Find the root of a simple 2 variables function.__

In [11]:
#import Pkg; Pkg.add("NLsolve")
using NLsolve

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


__Create a structure `Calibration` to hold the model parameters.__

In [14]:
struct Calibration
    α #capital share
    β #time discount (1/(1+r) where r=4%)
    γ #risk aversion or 1/EIS, EIS is calibrated from micro models
    δ #capital depreciation (from the data)
end

In [16]:
p = Calibration(0.3, 0.96, 4., 0.1)

Calibration(0.3, 0.96, 4.0, 0.1)

In [17]:
# DEfine a constructor with default values

Calibration(;α = 0.3, β =  0.96, γ = 4., δ = 0.1) = Calibration(α, β ,γ ,δ)

Calibration

__Define two functions:__
- `transition(z::Float64, k::Float64, p::Calibration)::Tuple{Float64}` which returns productivity and capital at date `t+1` as a function of productivity, capital and investment at date `t`
- `arbitrage(z::Float64, k::Float64, i::Float64, Z::Float64, K::Float64, I::Float64, p::Calibration)::Float64` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)


In [20]:
function transition(z, k, i , p)
    Z = p.ρ * z
    K = (1-p*ρ) * k +i
    return [Z, K]
end

function arbitrage(z, k, i, Z, K, I, p)
    y = exp(z) * k ^p.α
    c = y-i

    Y = exp(Z) * K^p.α
    C = Y-I

    optimality = p*β*(C/c)^(-p.γ)*(1-p.δ + p.α*exp(Z)*K^(p.α-1))-1

    return optimality
end


arbitrage (generic function with 2 methods)

__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments:__
- `arbitrage(s::Vector{Float64}, x::Vector{Float64}, S::Vector{Float64}, X::Vector{Float64}, p::Calibration)`
- `transition(s::Vector{Float64}, x::Vector{Float64}, p::Calibration)`

In [19]:
function transition(s,x,p)
    return transition(s[1], s[2], x[2], p )
end
function arbitrage(s, x, S, X, p)
    return arbitrage(s[1],s[2],x[2],S[1],S[2],X[2],p)
end

arbitrage (generic function with 2 methods)

__Write a function `steady_state(p::Calibration)::Tuple{Vector,Vector}` which computes the steady-state of the model computed by hand.__ It returns two vectors, one for the states, one for the controls. Check that the steady-state satisfies the model equations.


In [ ]:
function steady_state(p)
    K =(1/p.β- (1-p.δ))/p.α)^(1/p.α -1))
    i = K*p
    z=  0.


The first order system satisfies:
$$\begin{align}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{align}$$

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



__Write a function `first_order_model(s::Vector, x::Vector, p::Calibration)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`.__


__Write a function `T(X, M::PerturbedModel)`  which implements the time iteration step.__

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(1,2)` and check that the result satisfies the first order model.__

    

__Define two linear operators `L_S(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` and `L_T(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which implement the derivatives of the simulation and the time-iteration operator respectively.__

__Implement a function `spectral_radius(f::Function)::Float64` which implements the power iteration method to compute the biggest eigenvalues of the two previously defined operators. Check that Blamnchard-Kahn conditions are met.__

__Write a function `simulate(s0::Vector, X::Matrix, p::Calibration, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods (by using the formula $\Delta s_t = (E + F X) s_{t-11}$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

__Make some nice plots.__